# Data Science Take Home Test

## Initialization

Import libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time
start_time = time.time()

Load data

In [4]:
df_mes = pd.read_csv('I:\Javier Resano\Curriculum\Caso Carto\yellow_tripdata_2017-11.csv', sep=',')

## Data Exploration and Cleaning

In [5]:
df_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9284803 entries, 0 to 9284802
Data columns (total 17 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
RatecodeID               int64
store_and_fwd_flag       object
PULocationID             int64
DOLocationID             int64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 1.2+ GB


**To note:**
-  "tpep_pickup_datetime" and "tpep_dropoff_datetime" should be datetime
- PULocationID, DOLocationID, payment_type, RateCodeID are really numeric codes. I'll transform them to strings

In [6]:
df_mes['tpep_pickup_datetime'] = pd.to_datetime(df_mes['tpep_pickup_datetime'])
df_mes['tpep_dropoff_datetime'] = pd.to_datetime(df_mes['tpep_dropoff_datetime'])
df_mes['PULocationID'] = df_mes['PULocationID'].astype(str)
df_mes['DOLocationID'] = df_mes['DOLocationID'].astype(str)
df_mes['payment_type'] = df_mes['payment_type'].astype(str)
df_mes['RatecodeID'] = df_mes['RatecodeID'].astype(str)
df_mes['VendorID'] = df_mes['VendorID'].astype(str)

In [7]:
with pd.option_context('float_format', '{:.2f}'.format): print(df_mes.describe()) #10.294.628 cases

       passenger_count  trip_distance  fare_amount      extra    mta_tax  \
count       9284803.00     9284803.00   9284803.00 9284803.00 9284803.00   
mean              1.61           2.90        13.14       0.32       0.50   
std               1.26           3.73       129.61       0.45       0.06   
min               0.00           0.00      -499.00     -48.64      -0.50   
25%               1.00           0.97         6.50       0.00       0.50   
50%               1.00           1.60         9.50       0.00       0.50   
75%               2.00           3.00        15.00       0.50       0.50   
max             192.00         702.50    393221.50      69.80      96.00   

       tip_amount  tolls_amount  improvement_surcharge  total_amount  
count  9284803.00    9284803.00             9284803.00    9284803.00  
mean         1.89          0.34                   0.30         16.49  
std          2.62          1.98                   0.01        129.89  
min       -112.00        -14.75

** Findings. Data issues: **
- trip_distance sometimes is 0 (no travel?)
- fare_amount sometimes contains negative numbers
- extra sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.5 or 1 charges)
- mta_tax sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.5 charges)
- total_amount sometimes contains negative numbers
- tip_amount sometimes contains negative numbers
- tolls_amount sometimes contains negative numbers
- improvement_surcharge sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.3 charges)
   

Since we have a lot of data, I will remove wrong data. <br>
Initial number of rows:

In [8]:
df_mes.shape[0]

9284803

In [9]:
df_mes = df_mes[df_mes['trip_distance']>0]
df_mes.shape[0]

9224250

In [10]:
df_mes = df_mes[df_mes['fare_amount']>0]
df_mes.shape[0]

9219351

In [11]:
df_mes = df_mes[df_mes['extra']>=0]
df_mes.shape[0]

9219344

In [12]:
df_mes = df_mes[df_mes['mta_tax']>=0]
df_mes.shape[0]

9219344

In [13]:
df_mes = df_mes[df_mes['total_amount']>=0]
df_mes.shape[0]

9219344

In [14]:
df_mes = df_mes[df_mes['tip_amount']>=0]
df_mes.shape[0]

9219344

In [15]:
df_mes = df_mes[df_mes['tolls_amount']>=0]
df_mes.shape[0]

9219344

In [16]:
df_mes = df_mes[df_mes['improvement_surcharge']>=0]
df_mes.shape[0]

9219344

Now, I'll check and delete strange cases:

In [17]:
df_mes[(df_mes['improvement_surcharge']!=0.3)].shape[0]

501

In [18]:
df_mes = df_mes[df_mes['improvement_surcharge']==0.3]
df_mes.shape[0]

9218843

In [19]:
df_mes[(df_mes['mta_tax']!=0.5)&(df_mes['mta_tax']!=0)].shape[0]

0

In [20]:
df_mes[(df_mes['extra']!=0)&(df_mes['extra']!=0.5)&(df_mes['extra']!=1)&(df_mes['extra']!=1.5)].shape[0]

35442

In [21]:
df_mes[(df_mes['extra']!=0)&(df_mes['extra']!=0.5)&(df_mes['extra']!=1)&(df_mes['extra']!=1.5)].extra.unique()

array([4.50e+00, 6.98e+01, 6.42e+01, 2.00e-02, 1.01e+00, 6.00e-01,
       3.00e-01, 6.52e+00, 4.54e+00, 4.04e+01, 2.00e+00, 3.00e+00,
       3.50e+00, 7.50e-01, 8.00e-01])

In [22]:
df_mes = df_mes[(df_mes['extra']==0)|(df_mes['extra']==0.5)|(df_mes['extra']==1)|(df_mes['extra']==1.5)]
df_mes.shape[0]

9183401

All in all we have gone from 10294628 to 10180582 rows. We are keeping 98.9% of the initial dataset

### Feature Engineering
In order to improve dataset quality, I am going to create some new features based on the existing ones

1. Travel time

In [23]:
df_mes['travel_time'] = df_mes['tpep_dropoff_datetime'] - df_mes['tpep_pickup_datetime']
df_mes['travel_time'] = df_mes['travel_time'].dt.total_seconds()

2. Average speed

In [24]:
df_mes['average_speed'] = df_mes['trip_distance'] / df_mes['travel_time'] *3600

3. Hour of the day

In [25]:
df_mes['tpep_pickup_hour'] = pd.DatetimeIndex(df_mes['tpep_pickup_datetime']).hour
df_mes['tpep_dropoff_hour'] = pd.DatetimeIndex(df_mes['tpep_dropoff_datetime']).hour

4. Day of the week

In [26]:
df_mes['tpep_pickup_weekday'] = pd.DatetimeIndex(df_mes['tpep_pickup_datetime']).weekday
df_mes['tpep_dropoff_weekday'] = pd.DatetimeIndex(df_mes['tpep_dropoff_datetime']).weekday

And we check their values:

In [27]:
with pd.option_context('float_format', '{:.2f}'.format): print(df_mes.loc[:,['travel_time','average_speed']].describe()) #10.294.628 cases

       travel_time  average_speed
count   9183401.00     9183401.00
mean       1055.00            inf
std       15473.05            nan
min       -3551.00        -257.69
25%         413.00           6.89
50%         694.00           9.47
75%        1140.00          13.08
max    45466304.00            inf


Some new issues with data arise:
- There are some negative travel_time values
- "average_speed == inf" happens when travel_time is 0. Which mean there was no trip
- Very high average speeds (> 100 mph) also are a mark of wrong data

In [28]:
df_mes[(df_mes['average_speed'] == np.inf) | (df_mes['average_speed'] > 100)]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
168,1,2017-11-01 00:30:50,2017-11-01 00:31:44,1,5.00,5,N,265,265,1,...,30.00,0.00,0.3,90.30,54.0,3.333333e+02,0,0,2,2
794,2,2017-11-01 00:12:08,2017-11-01 00:12:12,1,0.22,5,N,264,260,1,...,8.82,0.00,0.3,44.12,4.0,1.980000e+02,0,0,2,2
857,2,2017-11-01 00:09:46,2017-11-01 00:09:49,1,1.17,1,N,264,246,1,...,1.46,0.00,0.3,8.76,3.0,1.404000e+03,0,0,2,2
1202,1,2017-11-01 00:05:23,2017-11-01 00:05:41,1,3.30,1,N,90,90,2,...,0.00,0.00,0.3,3.80,18.0,6.600000e+02,0,0,2,2
1454,1,2017-11-01 00:15:29,2017-11-01 00:16:11,1,4.60,5,N,265,265,1,...,9.15,10.50,0.3,54.95,42.0,3.942857e+02,0,0,2,2
3401,1,2017-11-01 00:01:22,2017-11-01 00:01:24,1,1.30,5,N,100,100,1,...,1.75,0.00,0.3,10.55,2.0,2.340000e+03,0,0,2,2
4120,1,2017-11-01 00:40:30,2017-11-01 00:40:52,1,0.90,5,N,49,49,1,...,2.00,0.00,0.3,29.00,22.0,1.472727e+02,0,0,2,2
6072,1,2017-11-01 00:29:17,2017-11-01 00:29:42,2,16.30,5,N,265,265,1,...,16.05,0.00,0.3,96.35,25.0,2.347200e+03,0,0,2,2
6518,1,2017-11-01 00:13:24,2017-11-01 00:13:53,1,16.80,5,N,265,265,1,...,10.00,0.00,0.3,138.30,29.0,2.085517e+03,0,0,2,2
7663,1,2017-11-01 00:30:51,2017-11-01 00:31:09,1,16.10,2,N,238,238,1,...,11.70,5.76,0.3,70.26,18.0,3.220000e+03,0,0,2,2


In [29]:
df_mes = df_mes[(df_mes['average_speed'] != np.inf) & (df_mes['average_speed'] < 100)]
df_mes.shape[0]

9175545

In [30]:
#I convert Timedelta value to float (number of seconds of the trip)
df_mes['travel_time'] = df_mes['travel_time'].astype('timedelta64[s]').dt.total_seconds()

In [31]:
df_mes[df_mes['travel_time'] < 0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
1383804,1,2017-11-05 01:54:17,2017-11-05 01:22:16,1,5.7,1,N,79,61,1,...,4.75,0.00,0.3,28.55,-1921.0,-10.681936,1,1,6,6
1383813,1,2017-11-05 01:48:35,2017-11-05 01:02:50,1,2.4,1,N,249,162,1,...,1.00,0.00,0.3,13.80,-2745.0,-3.147541,1,1,6,6
1383844,1,2017-11-05 01:59:21,2017-11-05 01:02:24,2,0.3,1,N,236,236,2,...,0.00,0.00,0.3,5.30,-3417.0,-0.316067,1,1,6,6
1383858,1,2017-11-05 01:55:30,2017-11-05 01:03:32,1,1.7,1,N,24,116,1,...,1.85,0.00,0.3,11.15,-3118.0,-1.962797,1,1,6,6
1383865,1,2017-11-05 01:51:17,2017-11-05 01:02:42,2,1.4,1,N,158,79,1,...,1.00,0.00,0.3,11.30,-2915.0,-1.728988,1,1,6,6
1383899,1,2017-11-05 01:37:43,2017-11-05 01:05:01,1,5.2,1,N,148,145,1,...,4.35,0.00,0.3,26.15,-1962.0,-9.541284,1,1,6,6
1383904,1,2017-11-05 01:58:58,2017-11-05 01:02:35,1,0.3,1,N,264,264,1,...,1.05,0.00,0.3,6.35,-3383.0,-0.319243,1,1,6,6
1383925,1,2017-11-05 01:52:49,2017-11-05 01:05:34,1,3.4,1,N,234,143,1,...,2.75,0.00,0.3,16.55,-2835.0,-4.317460,1,1,6,6
1383932,1,2017-11-05 01:57:20,2017-11-05 01:30:19,1,5.5,1,N,79,61,1,...,60.00,0.00,0.3,85.30,-1621.0,-12.214682,1,1,6,6
1383936,1,2017-11-05 01:58:31,2017-11-05 01:10:34,1,2.1,1,N,234,48,1,...,2.00,0.00,0.3,13.30,-2877.0,-2.627737,1,1,6,6


In [32]:
df_mes = df_mes[df_mes['travel_time'] > 0]
df_mes.shape[0]

9174132

All in all we have gone from 10294628 to 10171487 rows. We are still keeping **98.8% of the initial dataset**

### Filling out code variables
In order to encode all code variables (i.e. those that take string values, not numeric), and not miss any column if in the dataset a variable doesn't have a certain code, I am adding at the end of the dataframe new rows with all the vaules for these code varibles. I will remove these rows from the dataframe once they are encoded.

In [37]:
print(sorted(df_mes['payment_type'].unique()))

['1', '2', '3', '4']


In [38]:
print(sorted(df_mes['RatecodeID'].unique()))

['1', '2', '3', '4', '5', '6', '99']


In [39]:
df_mes['PULocationID'].unique().shape

(258,)

In [40]:
df_mes[df_mes['PULocationID'].astype('int64') >= 266]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday


In [41]:
df_mes['DOLocationID'].unique().shape

(262,)

In [42]:
df_mes[df_mes['DOLocationID'].astype('int64') >= 266]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday


All in all, we see that "payment_type" is mising 2 values (5 & 6), the locations also miss some out of the 265 listed in the document, and "RatecodeID" has an extra one: 99. As it is unknown for us, we wil get rid of the values (we can see there are only few of them)

In [43]:
df_mes[df_mes['RatecodeID'] == '99']

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
2561193,1,2017-11-08 21:21:51,2017-11-08 21:25:58,1,0.9,99,N,48,48,1,...,1.85,0.0,0.3,8.15,247.0,13.117409,21,21,2,2


In [44]:
df_mes = df_mes[df_mes['RatecodeID'] != '99']

## Model Building

I am importing model created for March

In [45]:
# Due to memory issues, I'll work with a smaller dataset.
df_mes2 = df_mes.sample(n=1000000, random_state=0) #.iloc[0:1000000,:]

In [46]:
shape1 = df_mes2.shape[0]
df2 = pd.DataFrame([['1'], ['2']], columns=(['VendorID']))
df3 = [str(x) for x in range(1,266)]
df3 = pd.DataFrame({'PULocationID':df3, 'DOLocationID':df3})
df4 = pd.DataFrame([['Y'], ['N']], columns=(['store_and_fwd_flag']))
df5 = [str(x) for x in range(1,7)]
df5 = pd.DataFrame({'payment_type':df5, 'RatecodeID':df5})

df_mes2 = df_mes2.append(df2, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df3, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df4, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df5, sort=False, ignore_index=True)

df_mes2.fillna(1, inplace=True) #1 is a value every string column has, so I am not adding new values here

In [47]:
df_mes2 = df_mes2.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
df_mes2 = pd.get_dummies(df_mes2, drop_first=True)

In [48]:
df_mes2 = df_mes2.head(shape1)

In [49]:
cols = list(df_mes2.columns)
cols.remove('tip_amount')

In [50]:
#Divide the dataset into input variables and output 
# (I get a memory error if I work with the whole set, so I'm reducing it to 1.000.000 rows)
X = df_mes2.loc[0:1000000,cols]
Y = df_mes2.loc[0:1000000,['tip_amount']]

### Model: Linear Regression
We start with a simple model in order to get a baseline to compare the rest

In [51]:
%store -r lin_reg

In [52]:
Y_lin_reg = lin_reg.predict(X)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y, Y_lin_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y, Y_lin_reg))

RMSE: 0.07536055230800988
Variance explained:  0.9990635594084737


This model explains 99.8% of the variance, and predicts the tip with a precission of around 9.7 cents (RMSE)

Another test:

In [53]:
start_time = time.time()

In [54]:
df_mes2 = df_mes.iloc[5000000:6000000,:]

shape1 = df_mes2.shape[0]
df2 = pd.DataFrame([['1'], ['2']], columns=(['VendorID']))
df3 = [str(x) for x in range(1,266)]
df3 = pd.DataFrame({'PULocationID':df3, 'DOLocationID':df3})
df4 = pd.DataFrame([['Y'], ['N']], columns=(['store_and_fwd_flag']))
df5 = [str(x) for x in range(1,7)]
df5 = pd.DataFrame({'payment_type':df5, 'RatecodeID':df5})

df_mes2 = df_mes2.append(df2, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df3, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df4, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df5, sort=False, ignore_index=True)

df_mes2.fillna(1, inplace=True) #1 is a value every string column has, so I am not adding new values here

df_mes2 = df_mes2.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
df_mes2 = pd.get_dummies(df_mes2, drop_first=True)

df_mes2 = df_mes2.head(shape1)

cols = list(df_mes2.columns)
cols.remove('tip_amount')

#Divide the dataset into input variables and output 
# (I get a memory error if I work with the whole set, so I'm reducing it to 1.000.000 rows)
X2 = df_mes2.loc[:,cols]
Y2 = df_mes2.loc[:,['tip_amount']]



In [55]:
Y_lin_reg = lin_reg.predict(X2)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y2, Y_lin_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y2, Y_lin_reg))

RMSE: 0.06792136760249438
Variance explained:  0.9992105019510663


In [56]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 22.715147495269775 seconds ---
